In [2]:
#pip install geopy

  Using cached geopy-2.3.0-py3-none-any.whl (119 kB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from geopy.distance import geodesic
import re

In [11]:
# 讀取事故資料作為df_a
df_a = pd.read_csv('KS_A1_CONBINE.csv')

# 讀取測站資料作為df_b
df_b = pd.read_csv('../../DEVIDE/氣象測站清單20230607.csv')

# 讀取測速資料作為df_c
df_c = pd.read_csv('../../DEVIDE/高雄市_camera.csv')

In [12]:
# 將測速資料的經緯度轉為數值
df_c['緯度'] = pd.to_numeric(df_c['緯度'], errors='coerce')
df_c['經度'] = pd.to_numeric(df_c['經度'], errors='coerce')

# 將經緯度為缺失值的那筆資料刪掉
df_c = df_c.dropna(subset=['緯度', '經度'])

In [13]:
# 日期的正規表達式
patternYears_month_date = r"(\d+)-(\d+)-(\d+)"

In [14]:
# 將日期的年月日分別切割之後放在list裡
def spilt_date(date):
    date_list = []
    match = re.search(patternYears_month_date, date)
    date_list.append(match.group(1))
    date_list.append(match.group(2))
    date_list.append(match.group(3))
    return date_list

# 該方法用來判斷該場車禍是否在指定測站的營運時間內，a、b、c值分別為六都車禍的日期、測站開始日期、測站撤銷日期
def during_activation_True_False(a, b, c):
    try:
        accident_date = spilt_date(a)
        station_start_date = spilt_date(b)
    
    except AttributeError:
        return_ture_false= False
        return return_ture_false
    
    if station_start_date[0]>accident_date[0]:
        after_start_date = False
        
    elif station_start_date[0] == accident_date[0]:
        
        if station_start_date[1]>accident_date[1]:
            after_start_date = False
            
        elif station_start_date[1] == accident_date[1]:
            
            if station_start_date[2] > accident_date[2]:
                after_start_date = False
                
            else:
                after_start_date = True
        else:
            after_start_date = True
                
    else:
        after_start_date = True
    
    if after_start_date:
        
        if pd.notna(c):
            station_stop_date = spilt_date(c)
            
            if accident_date[0] > station_stop_date[0]:
                return_ture_false = False
            
            elif accident_date[0] == station_stop_date[0]:
                
                if accident_date[1] > station_stop_date[1]:
                    return_ture_false = False
                    
                elif accident_date[1] == station_stop_date[1]:
                    
                    if accident_date[2] > station_stop_date[2]:
                        return_ture_false = False
                        
                    else: 
                        return_ture_false = True
                    
                else:
                    return_ture_false = True
                
            else: 
                return_ture_false = True
        
        else:
            return_ture_false= True
        
    else:
        return_ture_false= False
        
    return return_ture_false

# 對於df_a表中進來的每一筆資料，找到最接近的測站
def find_nearest_station():
    min_distance = float('inf')
    station_id = None
    for _, coord in df_b.iterrows():
        distance = geodesic((row['LATITUDE'], row['LONGITUDE']), (coord['緯度'], coord['經度'])).kilometers
        if distance < min_distance:
            if during_activation_True_False(row['WHOLE_DATE'], coord['資料起始日期'], coord['撤站日期']):
                min_distance = distance
                station_id = coord['站號']
            else:
                continue
    return station_id

# # 對於df_a表中進來的每一筆資料，找到最接近的測速，並限定範圍
def find_nearest_camera():
    min_distance = float('inf')
    camera_id = None
    limit_distance = 0.3
    for _, coord in df_c.iterrows():
        distance = geodesic((row['LATITUDE'], row['LONGITUDE']), (coord['緯度'], coord['經度'])).kilometers
        if distance < min_distance:
            min_distance = distance
            camera_id = coord['TEMP ID']
    if min_distance <= limit_distance:
        return camera_id

    else:
        return None

    

In [36]:
# # 建立空列
# new_column = pd.Series(dtype='str')

# # 在指定位置插入空的列
# df_a.insert(51, 'STATION_ID', new_column)
# df_a.insert(52, 'CAMERA_ID', new_column)

In [ ]:
第305036筆已新增完畢
第415848筆已新增完畢

In [15]:
# 執行一次就好(如果不小心重複執行，就看下面的程式執行到哪筆再把0修改為看到的最後一筆的數字)
stop_point = 0

In [ ]:
#A1 # 將方法找到的值填回df_a中(僅限第一次執行，途中有報錯、停止請除錯完在執行此程式的下一個程式)
# for index, row in df_a.iterrows():
#     df_a.loc[index, 'STATION_ID'] = find_nearest_station()
#     df_a.loc[index, 'CAMERA_ID'] = find_nearest_camera()
#     stop_point += 1
#     print(f'第{index}筆已新增完畢')



In [ ]:
#A1 # 上面遇到未知狀況停止，除錯完之後再執行這個程式
# for index, row in df_a.iterrows():
#     if index >= stop_point:
#         df_a.loc[index, 'STATION_ID'] = find_nearest_station()
#         df_a.loc[index, 'CAMERA_ID'] = find_nearest_camera()
#         stop_point += 1
#         print(f'第{index}筆已新增完畢')

In [ ]:
#A1 # 儲存結果到新的CSV檔案(檔名命名:縣市名稱+事故資料新增測站ID和測速ID+當下日期.csv)
# df_a.to_csv('KS_A2_CONBINE.csv', index=False, encoding= 'UTF-8-sig')

In [ ]:
for index, row in df_a.iterrows():
    df_a.loc[index, 'STATION_ID'] = find_nearest_station()
    df_a.loc[index, 'CAMERA_ID'] = find_nearest_camera()
     if index % 1000 == 0:
            df_a.to_csv('KS_A1_CONBINE.csv', index=False, encoding= 'UTF-8-sig')
    stop_point += 1
    print(f'第{index}筆已新增完畢')

In [16]:
for index, row in df_a.iterrows():
    if index >= stop_point:
        df_a.loc[index, 'STATION_ID'] = find_nearest_station()
        df_a.loc[index, 'CAMERA_ID'] = find_nearest_camera()
        if index % 1000 == 0:
            df_a.to_csv('KS_A1_CONBINE.csv', index=False, encoding= 'UTF-8-sig')
        stop_point += 1
        print(f'第{index}筆已新增完畢')

第0筆已新增完畢
第1筆已新增完畢
第2筆已新增完畢
第3筆已新增完畢
第4筆已新增完畢
第5筆已新增完畢
第6筆已新增完畢
第7筆已新增完畢
第8筆已新增完畢
第9筆已新增完畢
第10筆已新增完畢
第11筆已新增完畢
第12筆已新增完畢
第13筆已新增完畢
第14筆已新增完畢
第15筆已新增完畢
第16筆已新增完畢
第17筆已新增完畢
第18筆已新增完畢
第19筆已新增完畢
第20筆已新增完畢
第21筆已新增完畢
第22筆已新增完畢
第23筆已新增完畢
第24筆已新增完畢
第25筆已新增完畢
第26筆已新增完畢
第27筆已新增完畢
第28筆已新增完畢
第29筆已新增完畢
第30筆已新增完畢
第31筆已新增完畢
第32筆已新增完畢
第33筆已新增完畢
第34筆已新增完畢
第35筆已新增完畢
第36筆已新增完畢
第37筆已新增完畢
第38筆已新增完畢
第39筆已新增完畢
第40筆已新增完畢
第41筆已新增完畢
第42筆已新增完畢
第43筆已新增完畢
第44筆已新增完畢
第45筆已新增完畢
第46筆已新增完畢
第47筆已新增完畢
第48筆已新增完畢
第49筆已新增完畢
第50筆已新增完畢
第51筆已新增完畢
第52筆已新增完畢
第53筆已新增完畢
第54筆已新增完畢
第55筆已新增完畢
第56筆已新增完畢
第57筆已新增完畢
第58筆已新增完畢
第59筆已新增完畢
第60筆已新增完畢
第61筆已新增完畢
第62筆已新增完畢
第63筆已新增完畢
第64筆已新增完畢
第65筆已新增完畢
第66筆已新增完畢
第67筆已新增完畢
第68筆已新增完畢
第69筆已新增完畢
第70筆已新增完畢
第71筆已新增完畢
第72筆已新增完畢
第73筆已新增完畢
第74筆已新增完畢
第75筆已新增完畢
第76筆已新增完畢
第77筆已新增完畢
第78筆已新增完畢
第79筆已新增完畢
第80筆已新增完畢
第81筆已新增完畢
第82筆已新增完畢
第83筆已新增完畢
第84筆已新增完畢
第85筆已新增完畢
第86筆已新增完畢
第87筆已新增完畢
第88筆已新增完畢
第89筆已新增完畢
第90筆已新增完畢
第91筆已新增完畢
第92筆已新增完畢
第93筆已新增完畢
第94筆已新增完畢
第95筆已新增完畢
第96筆已新增完畢
第97筆已新增完畢
第98筆已新增完畢
第99筆已新增完畢
第100筆已新增完畢

In [17]:
df_a.to_csv('KS_A1_CONBINE.csv', index=False, encoding= 'UTF-8-sig')